# Bibliotecas

In [2]:
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#from transformers import AutoTokenizer, AutoModelForMaskedLM

import funcoes_br_berto as fbr
import funcoes_xlmr as fxlm


Downloading: 100%|██████████| 615/615 [00:00<00:00, 307kB/s]
Downloading: 100%|██████████| 4.83M/4.83M [00:01<00:00, 3.07MB/s]
Downloading: 100%|██████████| 8.68M/8.68M [00:01<00:00, 5.60MB/s]


## Configurações

In [3]:
# force CPU (make CPU visible)
cpus = tf.config.experimental.list_physical_devices('CPU')
print(cpus)
tf.config.set_visible_devices([], 'GPU')  # hide the GPU
tf.config.set_visible_devices(cpus[0], 'CPU') # unhide potentially hidden CPU
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

## Constantes

In [4]:
SEED = 42

### Leitura dos arquivos

In [5]:
arquivos = ['hn_balanceado', 'hn_desbalanceado', 'sn_balanceado', 'sn_desbalanceado']

lista_df = []
for arquivo in arquivos:

    df = pd.read_csv(f"Dados/Datasets/{arquivo}.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df.append(df)

In [6]:
#lista_df[0].dtypes

In [7]:
print(f"Tamanho dos Datasets:\n\n\t\t| Hard\t| Soft\t|\nBalanceado\t| {lista_df[0].shape[0]}\t| {lista_df[2].shape[0]}\t|\nDesbalanceado\t| {lista_df[1].shape[0]}\t| {lista_df[3].shape[0]}\t|")

Tamanho dos Datasets:

		| Hard	| Soft	|
Balanceado	| 8400	| 8400	|
Desbalanceado	| 13290	| 13290	|


# Experimentos Com Outros Métodos

## BERTo

In [8]:
lista_df_resultado = []
for nome, df in zip(arquivos, lista_df):
    
    X = df[["titulo_1", "titulo_2"]]#[df["titulo_1"].to_list(), df["titulo_2"].to_list()]
    y = df["match"].to_list()

    X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size = 0.3, random_state = SEED, stratify = y)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size = 0.2, random_state = SEED, stratify = y_train_valid)

    (nome, historico, y_test, y_pred) = fxlm.pipeline_bert(nome, X_train[:5], y_train[:5], X_valid[:5], y_valid[:5], X_test[:5], y_test[:5])

    pd.DataFrame.from_dict(historico.history).to_csv(f'Dados/Resultados/BERTo/{nome}_historico.csv', index = False)

    report = classification_report(y_test, y_pred, output_dict = True)
    df_resultado = pd.DataFrame(report).transpose()
    df_resultado['modelo'] = nome

    #df_resultado.to_csv(f'Dados/Resultados/BERTo/{nome}_resultado.csv', index = True)

    lista_df_resultado.append(df_resultado)

Downloading: 100%|██████████| 1.04G/1.04G [00:56<00:00, 19.7MB/s]
All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1/1 [==============================] - 25s 25s/step - loss: 0.6218 - accuracy: 1.0000 - val_loss: 0.6397 - val_accuracy: 0.8000
Epoch 2/3
1/1 [==============================] - 6s 6s/step - loss: 0.5844 - accuracy: 1.0000 - val_loss: 0.6276 - val_accuracy: 0.8000
Epoch 3/3
1/1 [==============================] - 6s 6s/step - loss: 0.5576 - accuracy: 1.0000 - val_loss: 0.6153 - val_accuracy: 0.8000


All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1/1 [==============================] - 21s 21s/step - loss: 0.6222 - accuracy: 1.0000 - val_loss: 0.6436 - val_accuracy: 0.8000
Epoch 2/3
1/1 [==============================] - 6s 6s/step - loss: 0.5763 - accuracy: 1.0000 - val_loss: 0.6240 - val_accuracy: 0.8000
Epoch 3/3
1/1 [==============================] - 7s 7s/step - loss: 0.4340 - accuracy: 1.0000 - val_loss: 0.5961 - val_accuracy: 0.8000


c:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
All PyTorch model weights were used when initializing T

Epoch 1/3
1/1 [==============================] - 23s 23s/step - loss: 0.6306 - accuracy: 0.8000 - val_loss: 0.6532 - val_accuracy: 0.8000
Epoch 2/3
1/1 [==============================] - 7s 7s/step - loss: 0.6210 - accuracy: 1.0000 - val_loss: 0.6348 - val_accuracy: 0.8000
Epoch 3/3
1/1 [==============================] - 7s 7s/step - loss: 0.5396 - accuracy: 1.0000 - val_loss: 0.6050 - val_accuracy: 0.8000


All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1/1 [==============================] - 21s 21s/step - loss: 0.6168 - accuracy: 1.0000 - val_loss: 0.6184 - val_accuracy: 0.8000
Epoch 2/3
1/1 [==============================] - 6s 6s/step - loss: 0.5815 - accuracy: 1.0000 - val_loss: 0.5829 - val_accuracy: 0.8000
Epoch 3/3
1/1 [==============================] - 7s 7s/step - loss: 0.4601 - accuracy: 1.0000 - val_loss: 0.4688 - val_accuracy: 0.8000


c:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       0.00      0.00      0.00         1

    accuracy                           0.80         5
   macro avg       0.40      0.50      0.44         5
weighted avg       0.64      0.80      0.71         5

[[4 0]
 [1 0]]


c:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
df_resultado

,precision,recall,f1-score,support,modelo
0,0.80,1.0,0.888889,4.0,sn_desbalanceado
1,0.00,0.0,0.000000,1.0,sn_desbalanceado
accuracy,0.80,0.8,0.800000,0.8,sn_desbalanceado
macro avg,0.40,0.5,0.444444,5.0,sn_desbalanceado
weighted avg,0.64,0.8,0.711111,5.0,sn_desbalanceado
